In [18]:
!pip install paho-mqtt

In [19]:
!pip install psycopg2-binary

In [20]:
 import ssl
import sys
import psycopg2
from sqlalchemy import create_engine
import paho.mqtt.client
import json

In [21]:
host=' lallah.db.elephantsql.com'
user ='okgedrkq'
password='OWJG1VssXGsfuUWTguGQoCmtZ5c5JZIp'
dbname='okgedrkq'

In [22]:
def select(query):
    cur = myConnection.cursor()
    try:
        cur.execute(query)
    except Exception as e:
        myConnection.commit()
        print('Error en el query:', e)
    else:
        records = cur.fetchall()
        cur.close()
        return records

In [24]:
myConnection = psycopg2.connect(host='lallah.db.elephantsql.com',
                                user ='okgedrkq', password='OWJG1VssXGsfuUWTguGQoCmtZ5c5JZIp',
                                dbname='okgedrkq')

# **Primer Query**

Cuales son los clientes conocidos y desconocidos que mas han comprado en cada una de las tiendas.


In [44]:
query = """
        WITH info AS (
          WITH Top3MiembroTienda1 AS (
            SELECT cliente.id AS Cliente, COUNT(factura.idcliente) AS N_Facturas, 'Miembro del club' AS club_clientes, 'Sucursal 1' AS sucursal
            FROM cliente
            INNER JOIN factura ON factura.idcliente = cliente.id
            WHERE idsucursal = 1 AND cliente.id IN (
              SELECT idcliente FROM club_clientes
            )
            GROUP BY Cliente
            ORDER BY N_Facturas DESC
            LIMIT 3
          ),
          Top3MiembroTienda2 AS (
            SELECT cliente.id AS Cliente, COUNT(factura.idcliente) AS N_Facturas, 'Miembro del club' AS club_clientes, 'Sucursal 2' AS sucursal
            FROM cliente
            INNER JOIN factura ON factura.idcliente = cliente.id
            WHERE idsucursal = 2 AND cliente.id IN (
              SELECT idcliente FROM club_clientes
            )
            GROUP BY Cliente
            ORDER BY N_Facturas DESC
            LIMIT 3
          ), 
          Top3DescTienda1 AS (
            SELECT cliente.id AS Cliente, COUNT(factura.idcliente) AS N_Facturas, 'Cliente Desconocido' AS club_clientes, 'Sucursal 1' AS sucursal
            FROM cliente
            INNER JOIN factura ON factura.idcliente = cliente.id
            WHERE idsucursal = 1 AND cliente.id NOT IN (
              SELECT idcliente FROM club_clientes
            )
            GROUP BY Cliente
            ORDER BY N_Facturas DESC
            LIMIT 3
          ),  
          Top3DescTienda2 AS (
            SELECT cliente.id AS Cliente, COUNT(factura.idcliente) AS N_Facturas, 'Cliente Desconocido' AS club_clientes, 'Sucursal 2' AS sucursal
            FROM cliente
            INNER JOIN factura ON factura.idcliente = cliente.id
            WHERE idsucursal = 2 AND cliente.id NOT IN (
              SELECT idcliente FROM club_clientes
            )
            GROUP BY Cliente
            ORDER BY N_Facturas DESC
            LIMIT 3
          )
          SELECT * FROM Top3MiembroTienda1
          UNION
          SELECT * FROM Top3MiembroTienda2
          UNION
          SELECT * FROM Top3DescTienda1
          UNION
          SELECT * FROM Top3DescTienda2
        )
        SELECT * FROM info
        ORDER BY Sucursal, club_clientes, N_Facturas DESC
"""
select(query)

[(4, 17, 'Miembro del club', 'Sucursal 1'),
 (1, 17, 'Miembro del club', 'Sucursal 1'),
 (7, 16, 'Miembro del club', 'Sucursal 1'),
 (2, 18, 'Miembro del club', 'Sucursal 2'),
 (10, 8, 'Miembro del club', 'Sucursal 2'),
 (8, 7, 'Miembro del club', 'Sucursal 2')]

## **Segundo Query**

Diga cual es la categoría de producto que menos se ha vendido en cada una de las tiendas.

In [43]:
query = """
      WITH info AS (
        WITH SucursalMenosVentas1 AS (
          SELECT productos.categoria as Categoria, SUM(ventas.cant) AS Ventas, sucursal.id 
          AS Sucursal FROM productos, ventas, sucursal
          WHERE productos.id = ventas.id_prod AND sucursal.id = 1
          GROUP BY Categoria, Sucursal
          ORDER BY Ventas ASC
          LIMIT 1
        ),
        SucursalMenosVentas2 AS (
          SELECT productos.categoria as Categoria, SUM(ventas.cant) AS Ventas, sucursal.id
          AS Sucursal FROM productos, ventas, sucursal
          WHERE productos.id = ventas.id_prod AND sucursal.id = 2
          GROUP BY Categoria, Sucursal
          ORDER BY Ventas ASC
          LIMIT 1
        )
        SELECT * FROM SucursalMenosVentas1
        UNION
        SELECT * FROM SucursalMenosVentas2
      )
      SELECT * FROM info
      ORDER BY Sucursal ASC
"""
select(query)      

[(2, 681, 1), (2, 681, 2)]

# **Tercer Query**

Cuales es el top 5 de productos que mas se han vendido en cada una de las tiendas.

In [45]:
query = """
        WITH info AS (
          WITH SucursalMasVentas1 AS (
            SELECT productos.nombre AS Nombre, SUM(ventas.cant) AS Ventas, sucursal.id
            AS Sucursal FROM productos, ventas, sucursal
            WHERE productos.id = ventas.id_prod AND sucursal.id = 1
            GROUP BY Nombre, Sucursal
            ORDER BY Ventas DESC
            LIMIT 5
          ),
          SucursalMasVentas2 AS (
            SELECT productos.nombre AS Nombre, SUM(ventas.cant) AS Ventas, sucursal.id
            AS Sucursal FROM Productos, ventas, sucursal
            WHERE productos.id = ventas.id_prod AND sucursal.id = 2
            GROUP BY Nombre, Sucursal
            ORDER BY Ventas DESC
            LIMIT 5
          )
          SELECT * FROM SucursalMasVentas1
          UNION
          SELECT * FROM SucursalMasVentas2
        )
        SELECT * FROM info
        ORDER BY Sucursal ASC, Ventas DESC
"""
select(query)

[('Macarrones con queso', 953, 1),
 ('Pescado', 929, 1),
 ('Oreos', 681, 1),
 ('Macarrones con queso', 953, 2),
 ('Pescado', 929, 2),
 ('Oreos', 681, 2)]

# **Cuarto Query**

Diga todos los clientes que en los últimos 7 días a partir de la ejecución del query han comprado solo en una tienda y los que han comprado en las dos tiendas.

In [41]:
#Clientes que solo han comprado en una tienda.

query = """
        WITH info AS (
          WITH Sucursal1 AS (
            SELECT factura.idcliente AS Cliente, factura.idsucursal AS SucursalVisitada FROM factura
            WHERE DATE_PART ('day', current_date::timestamp - factura.fecha::timestamp) <= 7
            AND factura.idsucursal = 1 
            AND factura.idcliente NOT IN (
              SELECT factura.idcliente AS Cliente FROM factura
                WHERE DATE_PART('day', current_date::timestamp - factura.fecha::timestamp) <= 7
                AND factura.idsucursal = 2
            ) 
          GROUP BY Cliente, SucursalVisitada
          ORDER BY Cliente

          ),

          Sucursal2 AS (
            SELECT factura.idcliente AS Cliente, factura.idsucursal AS SucursalVisitada FROM factura
            WHERE DATE_PART ('day', current_date::timestamp - factura.fecha::timestamp) <= 7
            AND factura.idsucursal = 2 
            AND factura.idcliente NOT IN (
              SELECT factura.idcliente AS Cliente FROM factura
                WHERE DATE_PART('day', current_date::timestamp - factura.fecha::timestamp) <= 7
                AND factura.idsucursal = 1
           )
         GROUP BY Cliente, SucursalVisitada
         ORDER BY Cliente

        )
        SELECT * FROM Sucursal1
        UNION 
        SELECT * FROM Sucursal2

     )
     SELECT * FROM info
     ORDER BY SucursalVisitada ASC, Cliente

"""
select(query)        

[]

In [40]:
#Clientes que han comprado en las dos tiendas. 

query = """
        WITH info AS (
          WITH Sucursales AS (
            SELECT factura.idcliente AS Cliente, factura.idsucursal AS SucursalesVisitadas FROM factura
            WHERE DATE_PART('day', current_date::timestamp - factura.fecha::timestamp) <= 7
            GROUP BY Cliente, SucursalesVisitadas
            ORDER BY Cliente, SucursalesVisitadas
          )
          SELECT Cliente, COUNT(SucursalesVisitadas) AS CantVisitadas FROM Sucursales
          GROUP BY Cliente 
        )
        SELECT Cliente FROM info 
        WHERE CantVisitadas = 2
        ORDER BY Cliente
"""        
select(query)

[(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,)]

# **Quinto Query**

Haga un análisis por estante de cuales categorías de producto tienen mayor rotación deinventario y haga una propuesta de modificación.

In [31]:
query = """
       WITH Propuesta AS (
          WITH info AS (
            SELECT productos.categoria AS Categoria, COUNT(restock_estantes.id_estante) AS CantRotacion, restock_estantes.fecha::date AS FechaRotacion
            FROM restock_estantes
            INNER JOIN estantes ON estantes.id = restock_estantes.id_estante
            INNER JOIN stock ON stock.id_producto = restock_estantes.id_producto
			      INNER JOIN productos ON productos.id = restock_estantes.id_producto
            WHERE estantes.id_sucursal = stock.id_sucursal
            GROUP BY Categoria, FechaRotacion
            ORDER BY FechaRotacion, Categoria
          )
          SELECT Categoria, CAST(AVG(CantRotacion) AS INT) AS RellenoxDia
          FROM info
          GROUP BY Categoria 
        )
        SELECT Categoria, RellenoxDia, CASE
              WHEN RellenoxDia > 2 THEN 'Estante con mucha rotacion, debe aumentarsele la capacidad.' 
              ELSE 'Estante con capacidad adecuada.'
              END AS Propuesta
        FROM Propuesta
        ORDER BY RellenoxDia DESC, Categoria 
"""
select(query)

[(1, 2, 'Estante con capacidad adecuada.'),
 (2, 2, 'Estante con capacidad adecuada.'),
 (3, 2, 'Estante con capacidad adecuada.')]

# **Sexto Query**

Encuentre a todos los clientes que han pagado con 2 bancos distintos en la ultima semana.
Enumérelos y diga si son parte del programa de afiliados.

In [46]:
query ="""
      WITH Cuentas AS (
        WITH Cantidad AS (
          SELECT idcliente AS Cliente, banco AS CantBancos FROM factura
          WHERE DATE_PART ('day', current_date::timestamp - factura.fecha::timestamp) <= 7
          GROUP BY Cliente, CantBancos
          ORDER BY Cliente, CantBancos
        )
        SELECT Cliente, COUNT(CantBancos) AS Cuentas FROM Cantidad
        GROUP BY Cliente
        ORDER BY Cuentas DESC

      ),
      ClientesClub AS(
        SELECT cliente.id AS Cliente, CASE
          WHEN cliente.id IN (
            SELECT id FROM club_clientes
            )THEN 'Miembro del Club' ELSE 'Desconocido'
          END AS ClientesClub
       FROM cliente     
      )

      SELECT Cuentas.Cliente, Cuentas.Cuentas AS CantBancos, ClientesClub.ClientesClub 
      FROM Cuentas, ClientesClub
      WHERE Cuentas >= 2 AND Cuentas.Cliente = ClientesClub.Cliente
"""
select(query)      


[(4, 3, 'Miembro del Club'),
 (3, 3, 'Miembro del Club'),
 (10, 3, 'Miembro del Club'),
 (9, 3, 'Miembro del Club'),
 (7, 3, 'Miembro del Club'),
 (1, 3, 'Miembro del Club'),
 (5, 3, 'Miembro del Club'),
 (2, 3, 'Miembro del Club'),
 (6, 3, 'Miembro del Club'),
 (8, 3, 'Miembro del Club')]